# Script with raster processing functions
### GEO 409, Fall 2018

Functions include: 
1. hillshade iterator that creates multiple images for evaluating which parameters look best
2. delete raster by file name pattern
3. print raster properties for all rasters in a workspace

![Furnace Arch](https://farm2.staticflickr.com/1973/31362521768_3e96f1b942_h.jpg)    
Furnace Arch, Kentucky

In [ ]:
# Import ArcPy package and set default environment properties and variables
import arcpy
arcpy.env.overwriteOutput = True

In [ ]:
# Ask for geodatabase to process
geodb = "Z:/BoydsGIS/data/rrg_build.gdb/" 
#geodb = input("Enter full pathname to geodatabase, using forward slashes: ")

# Show contents of database
try:
    arcpy.env.workspace = geodb
    rasterList = arcpy.ListRasters()
    print(f"Rasters available: {rasterList}")
except:
    print("Not a valid database")

In [ ]:
##########################################
#          Hillshade iterator            #
##########################################   
    
# Set default values for hillshade iterator function
in_raster = "dem_lo_res"
azimuth = 270
altitude = 55
shadows = "SHADOWS"
zFactor = 1

##########################################
#    Functions are defined here and      #
#   no variables should be set below     #
##########################################

# Hillshade iterator function
def hillshading(whichType, start, increment):
    """
    Hillshade function creates three hillshades.
    Parameters
    whichType: azimuth or altitude
    start: starting azimuth or altitude
    increment: amount to add in each iteration
    """
    try:
        # Reassigning these variables in function. 
        # The global keyword will allow function to use default values used in script. 
        global azimuth
        global altitude
        startNum = int(start)
        incrementNum = int(increment)
        if "az" in whichType.lower() and startNum + (2 * incrementNum) <= 360:
            i = startNum
            while i <= startNum + (2 * incrementNum):
                azimuth = i
                out_raster = "dem_hill_" + whichType.lower() + "_" + str(i)
                # f strings in 3.6 allow easier string concatentation
                print(f"Creating {out_raster}...")
                arcpy.HillShade_3d(in_raster, out_raster, azimuth, altitude, shadows, zFactor)
                i += incrementNum
        elif "al" in whichType.lower() and startNum + (2 * incrementNum) <= 90:
            i = startNum
            while i <= startNum + (2 * incrementNum):
                altitude = i
                out_raster = "dem_hill_" + whichType.lower() + "_" + str(i)
                print(f"Creating {out_raster}...")
                arcpy.HillShade_3d(in_raster, out_raster, azimuth, altitude, shadows, zFactor)
                i += incrementNum
        else:
            print("Check your azimuth (0-360) or altitude (0-90) ranges.")
    except:
        print("Check your input values.")
    print("All finished! Enjoy the 🌞")

##########################################
#       Get user input for iterator      #
########################################## 
# Call hillshade function inside another function.
def hillshadeWithInput():
    """Ask user for input and call hillshading function."""
    which_type = input("Iterate through azimuth or altitude:  ")
    start_value = input("Starting value: ")
    increment_value = input("Increment value: ")
    if which_type is "" or start_value is "" or increment_value is "":
        print("Missing values. Rerun.")
    else:
        try:
            print("Hillshading...")
            hillshading(which_type, start_value, increment_value)
        except:
            print("Check your input values.")
        
##########################################
#       Delete raster by pattern         #
########################################## 
# If no argument is supplies, the string "Temp" is passed.
def deleteRaster(prefix="Temp"):
    """The prefix parameter must not be empty and should be a string."""
    rasterList = arcpy.ListRasters()
    deleteList = []
    # Protect the database from complete deletion.
    for rl in rasterList:
        if prefix == "":
            print("%%%%%%\nYou'll delete everything!\n%%%%%%")
            break
        elif prefix in rl:
            deleteList.append(rl)
        else: 
            print(f"Keeping '{rl}'")
    if deleteList == []:
        return f"{prefix} not in a raster layers' name!"
    # Confirm deletion of layers
    deleteQuestion = input(f"Do you wish to delete these rasters? {deleteList}")
    if "y" in deleteQuestion.lower():
        for dl in deleteList:
            print(f"See ya later: '{geodb + dl}'")
            #Uncomment below to delete rasters
            arcpy.Delete_management(geodb + dl)
    else: 
        print("Ok. Nothing deleted.")
    
    rasterList = arcpy.ListRasters()
    print(f"All done! Rasters remaining: {rasterList}")

##########################################
#   Display properties of raster layers  #
##########################################     
# Use workspace defined at top
def rasterProps(databaseLocation=geodb):
    rasterList = arcpy.ListRasters()
    for rl in rasterList:
        desc = arcpy.Describe(rl)
        # Print some raster dataset properties
        print(f"Name: {rl}")
        print(f"Band Count: {desc.bandCount}")
        print(f"Raster Format: {desc.format}")
        print(f"Compression Type: {desc.compressionType}")
        print(f"Pixels: {desc.height}h x {desc.width}w")
        print(f"Pixel Type: {desc.pixelType}")
        print(f"Resolution: {desc.meanCellHeight}")
        print(f"Integer Raster: {desc.isInteger}")
        print("%%%%%%%%%%%%%%%%%%%%%%%%%%")


In [ ]:
##########################################
#      Functions are called below.       #
#          Uncomment to run              #
##########################################

# Call function directly   
# hillshading("azimuth", 200, 30)

# Call function to supply input and run
hillshadeWithInput() 

#deleteRaster()
#rasterProps()

This animation shows an example of visualizations that require Python scripting. 

* https://uky-gis.github.io/maps/rrg-summer-day/

![Animated GIF](https://outragegis.com/gorge/animations/sun.gif)